! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv
! pip install --upgrade azure-search-documents
!pip install --upgrade langchain
! pip install pandasql

In [12]:
# Import required libraries  
import os  
import json  
import openai  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
    VectorSearch, 
    HnswVectorSearchAlgorithmConfiguration,  
)  

from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import CharacterTextSplitter
from azure.storage.blob import BlobServiceClient
import tempfile
from langchain.document_loaders import PyPDFLoader
from azure.core.exceptions import HttpResponseError 
from langchain.embeddings.openai import OpenAIEmbeddings

In [3]:
# Configure environment variables  
#load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential("jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg")

In [4]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://testingchat.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e8143eabb02541259054426630db12a7"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
model: str = "text-embedding-ada-002"
#pdf_file_name = "Alaska Airlines and Horizon Air SMS Manual.pdf"     

AZURE_SEARCH_SERVICE_ENDPOINT = "https://genaisafety.search.windows.net"
AZURE_SEARCH_ADMIN_KEY = "jA50VnyT0OAeuHpY0RxJGz7z9kfrmdj7YF5PHYiPXjAzSeBS6hJg"

In [5]:
def create_index(AZURE_SEARCH_INDEX_NAME, fields):
    vector_search = VectorSearch(
        algorithm_configurations=[
            HnswVectorSearchAlgorithmConfiguration(
                name="my-vector-config",
                kind="hnsw",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            )
        ]
    )


    semantic_config = SemanticConfiguration(
        name="my-semantic-config",
        prioritized_fields=PrioritizedFields(
            title_field=SemanticField(field_name="id"),
            prioritized_keywords_fields=[SemanticField(field_name="content")]
        )
    )

    # Create the semantic settings with the configuration
    semantic_settings = SemanticSettings(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=AZURE_SEARCH_INDEX_NAME, fields=fields,
                        vector_search=vector_search, semantic_settings=semantic_settings)

    result = search_client.create_or_update_index(index)
    return result

In [6]:
#embeddings = OpenAIEmbeddings(deployment = model,chunk_size=1) 

# Replace with your actual connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

In [7]:
blobs = container_client.list_blobs()
dict_data = []
for blob in blobs:
    if blob.name.startswith("etq-data"):
        #blob_client = container_client.get_blob_client(blob)
        pdf_file_name=str(blob.name).split("/")[1] 
        bid=str(blob.name).split("/")[1].split(".")[0].lower().replace(" ","_").replace("_","-")
        print(bid)

vw-etq-events-fast
vw-reportit-allreport-fast


In [77]:
import pandas as pd
import io
from azure.storage.blob import BlobServiceClient

# Replace with your actual connection string
azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=storageaccountgenai;AccountKey=WAfyL4uqWgarFdx1ibNgWv9lmOINODLuN6nnLSQLgE/iuHhKGi1pYd6NQJ6LBnZO/DnnQfhbNSWi+AStsOLf1Q==;EndpointSuffix=core.windows.net"

# Replace with your container name
container_name = "safety"

try:
    # Connect to Azure Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # List all blobs in the container
    blobs = container_client.list_blobs()

    # Create an empty list to store DataFrame objects
    dfs = {}

    # Loop through the blobs and process those that match the prefix
    for blob in blobs:
        if blob.name.startswith("etq-data"):
            # Extract file name and bid name
            file_name = str(blob.name).split("/")[1]
            bid_name = str(blob.name).split("/")[1].split(".")[0].lower().replace(" ", "_").replace("_", "-")

            # Get the blob client for reading the blob data
            blob_client = container_client.get_blob_client(blob.name)
            print(blob.name)

            # Download the blob data as a stream
            blob_data = blob_client.download_blob()

            # Read the stream as a string and then into a Pandas DataFrame
            blob_text = blob_data.readall().decode('utf-8')
            df = pd.read_csv(io.StringIO(blob_text))

            # Append the DataFrame to the list
            dfs[file_name] = df

    # Now you can work with the list of DataFrames (dfs) as needed
    print(dfs.keys())

except Exception as e:
    print(f'Error: {str(e)}')


etq-data/vw-ETQ-Events-fast
etq-data/vw_ReportIt_AllReport_fast
dict_keys(['vw-ETQ-Events-fast', 'vw_ReportIt_AllReport_fast'])


In [78]:
dfs['vw-ETQ-Events-fast'].head(1)

,Division,INCIDENT_ID,EtqNumber,EventDate,Airline,Flight,Tail,Risk,ReportType,IrropType,...,RiskNotes,CreatedDate,Likelihood,Severity,ErrStation,DiscStation,EventLevel,DAGSRB,InvestReq,Time
0,Inflight,205,204,2016-10-28,NaN,2094,428QX,1,Inflight : Other Event,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN


In [79]:
dfs['vw_ReportIt_AllReport_fast'].head(1)

,Carrier,EventType,SubType,Division,ID,ObjectId,WeekStart,SubmittedDate,Title,Risk,...,Description,Category,EventDate,Recommendations,Linked_eventNumber,Source,ErrorStation,DiscoveredStation,PSID,DISPLAY_NAME
0,AS,Report,Irreg,Inflight,49020,49051,2018-01-01,2018-01-01 00:29:25,Inflight : Catering/Service Interruption,NaN,...,Was only catered with one beverage cart on a d...,Meal Issue,2017-12-31 12:00:00,NaN,39303.0,Mobile Application,NaN,NaN,1050875,Alexandra Prewett


In [111]:
# 1. Perform a left join between etq_df and ri_df
merged_df = pd.merge(dfs['vw-ETQ-Events-fast'], dfs['vw_ReportIt_AllReport_fast'], left_on='EtqNumber', right_on='Linked_eventNumber', how='left')
merged_df.head(1)

,Division_x,INCIDENT_ID,EtqNumber,EventDate_x,Airline,Flight_x,Tail_x,Risk_x,ReportType,IrropType,...,Description_y,Category,EventDate_y,Recommendations,Linked_eventNumber,Source,ErrorStation,DiscoveredStation,PSID,DISPLAY_NAME
0,Inflight,205,204,2016-10-28,NaN,2094,428QX,1,Inflight : Other Event,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
merged_df.columns

Index(['Division_x', 'INCIDENT_ID', 'EtqNumber', 'EventDate_x', 'Airline',
       'Flight_x', 'Tail_x', 'Risk_x', 'ReportType', 'IrropType', 'Cause',
       'CauseOther', 'Subject', 'Description_x', 'Origin', 'Destination',
       'Location_x', 'WSRDate', 'OPLEOI', 'AnalystNotes', 'RiskNotes',
       'CreatedDate', 'Likelihood', 'Severity', 'ErrStation', 'DiscStation',
       'EventLevel', 'DAGSRB', 'InvestReq', 'Time', 'Carrier', 'EventType',
       'SubType', 'Division_y', 'ID', 'ObjectId', 'WeekStart', 'SubmittedDate',
       'Title', 'Risk_y', 'Location_y', 'Tail_y', 'Flight_y', 'Description_y',
       'Category', 'EventDate_y', 'Recommendations', 'Linked_eventNumber',
       'Source', 'ErrorStation', 'DiscoveredStation', 'PSID', 'DISPLAY_NAME'],
      dtype='object')

In [113]:
# 2. Apply the filtering conditions
event_start_date = '2023-01-01'
event_end_date = '2023-02-01'

filtered_df = merged_df[(merged_df['EventDate_x'] >= event_start_date) &
                        (merged_df['EventDate_x'] < event_end_date) &
                        (merged_df['Recommendations'].notnull())]

# 3. Select the desired columns
selected_columns = ['ID','Division_x', 'EventDate_x', 'Subject', 'Description_x', 'Recommendations']
result_df = filtered_df[selected_columns]
result_df.head(1)

,ID,Division_x,EventDate_x,Subject,Description_x,Recommendations
191392,407297.0,Inflight,2023-01-15,Pax travelling with PETC; not compliant with p...,- Report # 407297\n \nWendy Boyer in 20C had h...,Give her a warning. If she does it again she ...


In [141]:
# Determine the total number of rows in the DataFrame
total_rows = len(result_df)

# Calculate the number of rows in each half
rows_per_half = total_rows // 200

In [142]:
# Create slices of the DataFrame for each half
dataframes_list = []
data_dict_list = []
partitionid = []
ID = []
Division = []
EventDate = []
Subject = []
Description = []
Recommendations = []
for i in range(200):
    start_row = i * rows_per_half
    end_row = (i + 1) * rows_per_half
    df_slice = result_df.iloc[start_row:end_row]
    dataframes_list.append(df_slice)
    #convert DF to JSON
    json_data = df_slice.to_json()
    data_dict = json.loads(json_data)
    data_dict_list.append(data_dict)
    #partitionID
    partitionid.append(i)
    #other fields
    ID.append(json.dumps(data_dict['ID']))
    Division.append(json.dumps(data_dict['Division_x']))
    EventDate.append(json.dumps(data_dict['EventDate_x']))
    Subject.append(json.dumps(data_dict['Subject']))
    Description.append(json.dumps(data_dict['Description_x']))
    Recommendations.append(json.dumps(data_dict['Recommendations']))
    

In [176]:
partitionid

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [203]:
AZURE_SEARCH_INDEX_NAME=bid_name + "-index"
print(AZURE_SEARCH_INDEX_NAME)

search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)


vw-reportit-allreport-fast-index


In [204]:
AZURE_SEARCH_INDEX_NAME 

'vw-reportit-allreport-fast-index'

In [205]:
try:
    existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
except Exception as e:
    existing_index = None
    # The index doesn't exist, so create a new one
    
metadata = result_df.columns.tolist()
if existing_index == None:
    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="Division", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="EventDate", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="subject", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="description", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="Recommendations", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
    ]

    result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
    print(f' {result.name} created index')                        
    final_doc=[]
    for index in range(len(partitionid)):
        final_doc.append({'id': str(partitionid[index]), 
                          'Division': Division[index],
                          'EventDate': EventDate[index],  #df.iloc[index],
                          'subject': Subject[index],
                          'description': Description[index],
                          'Recommendations': Recommendations[index],
                          'content': str(data_dict_list[index]),
                          'metadata': str(metadata),
                         })

     
    # Define a list to store batches of documents
    #document_batches = [final_doc[i] for i in range(0, len(final_doc))]

    # Upload documents in batches
    for batch in final_doc:
        try:
            result = search_client_v2.upload_documents(batch)
            print('documents uploaded')
        except Exception as e:
            print(f"Error uploading documents: {e}")
            print(repr(e))
            break

    print("Vector store added successfully")

else:
    try:
        # Check if the index exists
        # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        final_doc=[]
        for index in range(len(partitionid)):
            final_doc.append({'id': str(partitionid[index]), 
                              'Division': Division[index],
                              'EventDate': EventDate[index],  #df.iloc[index],
                              'subject': Subject[index],
                              'description': Description[index],
                              'Recommendations': Recommendations[index],
                              'content': str(data_dict_list[index]),
                              'metadata': str(metadata),
                             })

        

        # Define a list to store batches of documents
        document_batches = [final_doc[i] for i in range(0, len(final_doc))]

        # Upload documents in batches
        for batch in document_batches:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}") 
                print(repr(e))
                break

        print("Vector store added successfully")


    except Exception as e:
        print(e)
        pass

 vw-reportit-allreport-fast-index created index
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
do

New work

In [206]:
AZURE_SEARCH_INDEX_NAME=bid_name + "-wc-index"
print(AZURE_SEARCH_INDEX_NAME)


search_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
search_client_v2 = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)


vw-reportit-allreport-fast-wc-index


In [207]:
try:
    existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
except Exception as e:
    existing_index = None
    # The index doesn't exist, so create a new one
    
metadata = result_df.columns.tolist()
if existing_index == None:
    fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, searchable=True),
    SearchableField(name="metadata", type=SearchFieldDataType.String, searchable=True),
    ]

    result = create_index(AZURE_SEARCH_INDEX_NAME, fields)
    print(f' {result.name} created index')                        
    final_doc=[]
    for index in range(len(partitionid)):
        final_doc.append({'id': str(partitionid[index]), 
                          'content': str(data_dict_list[index]),
                          'metadata': str(metadata),
                         })

     
    # Define a list to store batches of documents
    #document_batches = [final_doc[i] for i in range(0, len(final_doc))]

    # Upload documents in batches
    for batch in final_doc:
        try:
            result = search_client_v2.upload_documents(batch)
            print('documents uploaded')
        except Exception as e:
            print(f"Error uploading documents: {e}")
            print(repr(e))
            break

    print("Vector store added successfully")

else:
    try:
        # Check if the index exists
        # existing_index = search_client.get_index(AZURE_SEARCH_INDEX_NAME)
        # print(f"Index '{AZURE_SEARCH_INDEX_NAME}' already exists. You can use the existing index.")
        final_doc=[]
        for index in range(len(partitionid)):
            final_doc.append({'id': str(partitionid[index]), 
                              'content': str(data_dict_list[index]),
                              'metadata': str(metadata),
                             })

        

        # Define a list to store batches of documents
        document_batches = [final_doc[i] for i in range(0, len(final_doc))]

        # Upload documents in batches
        for batch in document_batches:
            try:
                result = search_client_v2.upload_documents(batch)
                print('documents uploaded')
            except Exception as e:
                print(f"Error uploading documents: {e}") 
                print(repr(e))
                break

        print("Vector store added successfully")


    except Exception as e:
        print(e)
        pass

 vw-reportit-allreport-fast-wc-index created index
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded
documents uploaded